In [4]:
import os
from pathlib import Path
import numpy as np
import torch
from RBM_helper import RBM
import csv
import Rydberg_energy_calculator
from datetime import datetime
import ipyparallel as ipp

rc = ipp.Client()

{0: 92478, 1: 92461, 2: 92475, 3: 92487, 4: 92484, 5: 92490, 6: 92481, 7: 92493, 8: 92499, 9: 92496, 10: 92502, 11: 92462, 12: 92466, 13: 92472, 14: 92463, 15: 92469}


In [ ]:
training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt"))
n_vis = training_data.shape[1]

target = 0.0001
exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)
output_dir = Path("training_logs")

if not output_dir.exists():
    output_dir.mkdir(parents=True)
differences = {}
training_num = training_data.shape[0]
print(training_num)

In [ ]:
target = 0.0001
exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)
output_dir = Path("training_logs")
if not output_dir.exists():
    output_dir.mkdir(parents=True)
differences = {}
n_hin = 20
for i in range(19):
    train_num = 500 + i * 1000
    print("Training samples: ", train_num)
    sub_training_data = training_data[:train_num]
    rbm = RBM(n_vis, n_hin)
    epochs = 500
    num_samples = 2000
    min_difference = 10000
    min_difference_epoch = 0
    print('Starting training on n_h = ', n_hin)
    differences[n_hin] = []
    for e in range(1, epochs+1):
        # do one epoch of training
        rbm.train(training_data)

        # now generate samples and calculate the energy
        if e % 100 == 0:
            print("\nEpoch: ", e)
            # print("Sampling...")

            init_state = torch.zeros(num_samples, n_vis)
            RBM_samples = rbm.draw_samples(100, init_state)

            # print("Done sampling. Calculating energy...")

            energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction)
            energy_difference = abs(exact_energy - energies.item())
            differences[n_hin].append(energy_difference)
            print("Energy difference: ", energy_difference)
            if min_difference > energy_difference:
                min_difference = energy_difference
                min_difference_epoch = e
            if  min_difference < target:
                print("Target energy difference obtained at epoch = ", e)
                print("Final energy difference for n_h = ", n_hin, ' : ', energy_difference)
                break


    print("Final energy difference for n_h = ", n_hin, ' : ', min_difference, 'at epoch ', min_difference_epoch)
    if min_difference < target:
        print("Required samples: ", num_samples)
    with open(str(output_dir / f"task2_2_train_num_{train_num}.csv"), 'w') as f:
        w = csv.writer(f)
        w.writerow(differences[n_hin])
